In [ ]:
import os
import random
import pathlib
import subprocess

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
khm_training_text_files = r'/content/drive/MyDrive/ABA_BANK/teseract_fine-tuning/langdata_lstm/khm/khm.training_text'
fontconf_dir = '/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/fonts.conf'
output_directory = r'/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain/khm_data/khm-ground-truth'
training_text_files = r'/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/langdata/eng.training_text'

## Adding Font into collab environment path

In [ ]:
!cp "/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/fonts/FontsFree-Net-Apex-Regular.ttf" /usr/share/fonts/truetype/

In [ ]:
!fc-cache -fv
!fc-list | grep "Apex"

/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 5 fonts, 2 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded
/usr/share/fonts/truetype/FontsFree-Net-Apex-Regular.ttf: Apex:style=Regular


## Generating Text Line image with box ( For Training )

In [ ]:
lines = []

with open(training_text_files, 'r', encoding='utf-8') as input_file:
    for line in input_file.readlines():
        lines.append(line.strip())

In [ ]:
if not os.path.exists(output_directory):
    os.makedirs(output_directory)


random.shuffle(lines)
count = 10
lines = lines[:count]

line_count = 0
for line in lines:

    training_text_file_name = pathlib.Path(training_text_files).stem
    line_training_text = os.path.join(output_directory, f'{training_text_file_name}_{line_count}.gt.txt')

    # Write the text line to the ground truth text file
    with open(line_training_text, 'w') as output_file:
        output_file.write(line.strip() + '\n')

    file_base_name = f'eng_{line_count}'

    try:
        subprocess.run([
            'text2image',
            f'--fontconfig_tmpdir=(fontconf_dir)',
            '--font=Apex',
            f'--text={line_training_text}',
            f'--outputbase={os.path.join(output_directory, file_base_name)}',
            '--max_pages=1',
            '--strip_unrenderable_words',
            '--leading=32',
            '--xsize=3600',
            '--ysize=480',
            '--char_spacing=1.0',
            '--exposure=0',
            '--unicharset_file=/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/langdata/eng.unicharset'
        ], check=True)
    except subprocess.CalledProcessError as e:
        print(f'An error occurred while running text2image for line {line_count}: {e}')
    else:
        print(f'Generated image for line {line_count}')

    line_count += 1


Generated image for line 0
Generated image for line 1
Generated image for line 2
Generated image for line 3
Generated image for line 4
Generated image for line 5
Generated image for line 6
Generated image for line 7
Generated image for line 8
Generated image for line 9


## Fine-Tuning

In [ ]:
%cd /content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain

/content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesstrain


In [ ]:
!ls

data			       generate_wordstr_box.py	ocrd-testset.zip  shuffle.py
generate_gt_from_box.py        LICENSE			plot		  src
generate_line_box.py	       Makefile			README.md
generate_line_syllable_box.py  normalize.py		requirements.txt


In [ ]:
!make training MODEL_NAME=Apex START_MODEL=eng TESSDATA=../content/drive/MyDrive/ABA_BANK/tesseract_tutorial/tesseract/tessdata MAX_ITERATIONS=10000

    You are using make version: 4.3
make: *** No rule to make target 'data/Apex-ground-truth/eng_10.lstmf', needed by 'data/Apex/all-lstmf'.  Stop.


In [ ]:
#The original cmd from the vdo
TESSDATA_PREFIX=../tesseract/tessdata make training MODEL_NAME=Apex START_MODEL=eng TESSDATA=../tesseract/tessdata MAX_ITERATIONS=10000